# Example Temporal Logic Monitoring in Python

RTAMT is a Python library for monitoring Signal Temporal Logic (STL). It supports both offline and online monitoring of discrete-time and dense-time STL specifications. This tutorial provides a step-by-step guide to installing and using RTAMT.

## Installation

Install RTAMT as a Python package:

In [ ]:
%pip install rtamt

## Usage

### Dense-Time Offline Monitor
To perform offline monitoring of dense-time STL specifications (example from https://link.springer.com/article/10.1007/s10009-023-00720-3):

In [ ]:
import rtamt

# Signals are lists of tuples (timestamp, value)
req = [[0.0, 0.0], [3.0, 6.0], [5.0, 0.0]]
gnt = [[0.0, 0.0], [7.0, 6.0], [9.0, 0.0]]

spec = rtamt.StlDenseTimeSpecification()
# Declare the variables that will correspond to the above signals.
spec.declare_var('req', 'float')
spec.declare_var('gnt', 'float')
spec.spec = 'always((req >= 3) implies (eventually[0:5](gnt >= 3)))'
spec.parse()

# Evaluate the specification and match the variables with the signals.
rob = spec.evaluate(['req', req], ['gnt', gnt])

print('Robustness:', rob)

Note that:
 1. The output format is [[timestamp, robustness]]
 2. RTAMT only computes the quantitative/robustness semantics

For more details, refer to the [RTAMT GitHub repository](https://github.com/nickovic/rtamt).


## Exercises

### 1: Quantitative v.s. Boolean Semantics

Can you convert the output from `spec.evaluate()` into boolean verdicts? (i.e. an array of form `[[float, bool]]`)

Compare this to the quntitative results. Which gives you the most information?

### 2: Streaming input

RTAMT also has an online monitoring algorithm, which receives streams incrementally.

To use this create the spec with the `rtamt.StlDenseTimeOnlineSpecification()` method and provide each value with the method `spec.update` method, which takes a list of stream values, and returns a single robustness value.

Try running the following supecification:

In [ ]:
import sys
import rtamt

def monitor():
    a1 = [(0, 3), (3, 2)]
    b1 = [(0, 2), (2, 5), (4, 1), (7, -7)]

    a2 = [(5, 6), (6, -2), (8, 7), (11, -1)]
    b2 = [(10, 4)]

    a3 = [(13, -6), (15, 0)]
    b3 = [(15, 0)]

    # # stl
    spec = rtamt.StlDenseTimeSpecification()
    spec.name = 'STL dense-time specification'
    spec.declare_var('a', 'float')
    spec.spec = 'a>=2'
    try:
        spec.parse()
    except rtamt.RTAMTException as err:
        print('RTAMT Exception: {}'.format(err))
        sys.exit()

    rob = spec.update(['a', a1], ['b', b1])
    print('rob: ' + str(rob))

    rob = spec.update(['a', a2], ['b', b2])
    print('rob: ' + str(rob))

    rob = spec.update(['a', a3], ['b', b3])
    print('rob: ' + str(rob))

if __name__ == '__main__':
    monitor()


#### 2.1: Streams of inputs

How does the `update` method allow monitors to handle progressively arriving data?

Think about how you could use this to monitor data received from a network or a time-series database?


### 3: More info

Have a look at the description of the tool in https://link.springer.com/article/10.1007/s10009-023-00720-3 and the [RTAMT GitHub repository](https://github.com/nickovic/rtamt).

What other types of useful monitors can you write in RTAMT?